The goal of this kernel is to train a simple transformer from only pytorch to classify hate speech in comments
to understand how the transformer works, I would recommend this video it walks through the Attention paper and explains it well

https://www.youtube.com/watch?v=U0s0f995w14&t=2522s

I will be using  torchtext it's really good for preparing the data, it has a good documentation also 
those are links about an example using torchtext and a tutorial

https://www.analyticsvidhya.com/blog/2020/01/first-text-classification-in-pytorch/

http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/

as this will be a classification task, we will just need the encoder part of the transformer, so I used the transformer encoder layer from pytorch

In [1]:
import random

SEED = 32
random.seed(SEED)

import numpy as np 
import pandas as pd
import spacy

from sklearn.metrics import accuracy_score
from sklearn.metrics import  f1_score

from torch import nn
import torch
from torchtext import data
from torch.nn  import functional as F
import torch.optim as  optim 
if torch.cuda.is_available():  
  dev = "cuda:0" 
  print("gpu up")
else:  
  dev = "cpu"  
device = torch.device(dev)

gpu up


In [2]:
"""

those are the libraries I use for processing text

"""

import nltk
nltk.download("punkt")

import re
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()

tokenizer = Tokenizer(nlp.vocab)

from nltk import word_tokenize,sent_tokenize
from nltk.stem  import PorterStemmer


from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nltk.download('stopwords')
stops = stopwords.words("english")


def removepunc(my_str): # function to remove punctuation
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))
snowstem = SnowballStemmer("english")
portstem = PorterStemmer()


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
traindata = pd.read_csv("/kaggle/input/hate-speech-detection/toxic_train.csv")
test = pd.read_csv("/kaggle/input/hate-speech-detection/toxic_test.csv")
traindata.drop("Unnamed: 0",axis=1,inplace=True)
test.drop("Unnamed: 0",axis=1,inplace=True)

In [4]:
"""
this function is the tokenizer we are using, it does basic processing also  like ,
Lowercase the text
removing punctuation, stop words and numbers,
it also removes extra spaces and unwanted characters (I use regex for that)


before using the tokenizer I was testing it on the train dataframe manually  
"""

def myTokenizer(x):
 return  [snowstem.stem(word.text)for word in 
          tokenizer(removepunc(re.sub(r"\s+\s+"," ",re.sub(r"[^A-Za-z0-9()!?\'\`\"\r+\n+]"," ",x.lower()))).strip()) 
          if (word.text not in stops and not hasNumbers(word.text)) ]


In [5]:
"""
here I'm using the torchtext fields and dataset classes they can ease the work to get
the dataset ready for the pytorch model

the class DataFrameDataset is the easiest way I found to turn a dataframe into a torchtext dataset

this cell will take sometime to finish
"""

TEXT = data.Field(tokenize=myTokenizer,batch_first=True,fix_length=140)
LABEL = data.LabelField(dtype=torch.float ,batch_first=True)


class DataFrameDataset(data.Dataset):

    def __init__(self, df, text_field, label_field, is_test=False, **kwargs):
        fields = [('comment_text', text_field), ('toxic', label_field)]
        examples = []
        for i, row in df.iterrows():
            label = row.toxic 
            text = row.comment_text
            examples.append(data.Example.fromlist([text, label], fields))

        super().__init__(examples, fields, **kwargs)
  

torchdataset = DataFrameDataset(traindata, TEXT,LABEL)
torchtest = DataFrameDataset(test, TEXT,LABEL)

In [6]:
train_data, valid_data = torchdataset.split(split_ratio=0.8, random_state = random.seed(SEED))

In [7]:
"""
this cell build the vocab which means it get all the used words and if also ignores any word 
that only appeared less than 3 times
"""
TEXT.build_vocab(train_data,min_freq=3)  
LABEL.build_vocab(train_data)


In [8]:
#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  


Size of TEXT vocabulary: 38591
Size of LABEL vocabulary: 2
[('articl', 59408), ('page', 45801), ('wikipedia', 38257), ('edit', 33298), ('talk', 32102), ('use', 28301), ('one', 24706), ('like', 24462), ('pleas', 23766), ('would', 23436)]


In [9]:
#set batch size
BATCH_SIZE = 128

"""
we are using batches for validation and test set because of memory usage we can't pass the whole set at once 
"""


train_iterator,valid_iterator,test_iterator= data.BucketIterator.splits(
    (train_data,valid_data,torchtest), 
    batch_size = BATCH_SIZE,
    device = device,
    sort =False,
shuffle=False)


In [10]:

"""
one major point here is that I encoded the embeddings in a different way 
I made an embedding layer for the position then I concatenated position embeddings with the word embeddings 
just thought it could be a usefull way to encode the positions 

had to reshape the output of the transformer layer to get the prediction
"""
class TextTransformer(nn.Module):
  def __init__(self):
    super(TextTransformer,self).__init__()
    self.wordEmbeddings = nn.Embedding(len(TEXT.vocab),140)
    self.positionEmbeddings = nn.Embedding(140,20)
    self.transformerLayer = nn.TransformerEncoderLayer(160,8) 
    self.linear1 = nn.Linear(160,  64)
    self.linear2 = nn.Linear(64,  1)
    self.linear3 = nn.Linear(140,  16)
    self.linear4 = nn.Linear(16,  1)
  def forward(self,x):
    positions = (torch.arange(0,140).reshape(1,140) + torch.zeros(x.shape[0],140)).to(device) 
    # broadcasting the tensor of positions 
    sentence = torch.cat((self.wordEmbeddings(x.long()),self.positionEmbeddings(positions.long())),axis=2)
    attended = self.transformerLayer(sentence)
    linear1 = F.relu(self.linear1(attended))
    linear2 = F.relu(self.linear2(linear1))
    linear2 = linear2.view(-1,140) # reshaping the layer as the transformer outputs a 2d tensor (or 3d considering the batch size)
    linear3 = F.relu(self.linear3(linear2))
    out = torch.sigmoid(self.linear4(linear3))
    return out

myTransformer = TextTransformer()
myTransformer.to(device)

    


TextTransformer(
  (wordEmbeddings): Embedding(38591, 140)
  (positionEmbeddings): Embedding(140, 20)
  (transformerLayer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): Linear(in_features=160, out_features=160, bias=True)
    )
    (linear1): Linear(in_features=160, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=160, bias=True)
    (norm1): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (linear1): Linear(in_features=160, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=1, bias=True)
  (linear3): Linear(in_features=140, out_features=16, bias=True)
  (linear4): Linear(in_features=16, out_features=1, bias=True)
)

In [11]:
def calculateMetrics(ypred,ytrue):
  acc  = accuracy_score(ytrue,ypred)
  f1  = f1_score(ytrue,ypred)
  f1_average  = f1_score(ytrue,ypred,average="macro")
  return " f1 score: "+str(round(f1,3))+" f1 average: "+str(round(f1_average,3))+" accuracy: "+str(round(acc,3))
  

In [12]:
"""
using adagrad because it assign bigger updates to less frequently updated weights 
(like words that are not used many times)

"""

optimizer = optim.Adagrad(myTransformer.parameters(),lr = 0.001)

for i in range(20):
  trainpreds = torch.tensor([])
  traintrues = torch.tensor([])
  for  batch in train_iterator:
    X = batch.comment_text
    y = batch.toxic
    myTransformer.zero_grad()
    pred = myTransformer(X).squeeze()
    trainpreds = torch.cat((trainpreds,pred.cpu().detach()))
    traintrues = torch.cat((traintrues,y.cpu().detach()))
    err = F.binary_cross_entropy(pred,y)
    err.backward()
    optimizer.step()
  err = F.binary_cross_entropy(trainpreds,traintrues)
  print("train BCE loss: ",err.item(),calculateMetrics(torch.round(trainpreds).numpy(),traintrues.numpy()))
 

  valpreds = torch.tensor([])
  valtrues = torch.tensor([])
  for batch in valid_iterator:
    X = batch.comment_text
    y = batch.toxic
    valtrues = torch.cat((valtrues,y.cpu().detach()))
    pred = myTransformer(X).squeeze().cpu().detach()
    # print(valtrues.shape)
    valpreds = torch.cat((valpreds,pred))
  err = F.binary_cross_entropy(valpreds,valtrues)
  print("validation BCE loss: ",err.item(),calculateMetrics(torch.round(valpreds).numpy(),valtrues.numpy()))
  

train BCE loss:  0.3100277781486511  f1 score: 0.002 f1 average: 0.476 accuracy: 0.903
validation BCE loss:  0.2964196801185608  f1 score: 0.001 f1 average: 0.475 accuracy: 0.905
train BCE loss:  0.27820736169815063  f1 score: 0.007 f1 average: 0.478 accuracy: 0.904
validation BCE loss:  0.25836822390556335  f1 score: 0.018 f1 average: 0.484 accuracy: 0.905
train BCE loss:  0.2364715188741684  f1 score: 0.164 f1 average: 0.559 accuracy: 0.911
validation BCE loss:  0.22028805315494537  f1 score: 0.285 f1 average: 0.621 accuracy: 0.918
train BCE loss:  0.20641660690307617  f1 score: 0.404 f1 average: 0.682 accuracy: 0.924
validation BCE loss:  0.19979296624660492  f1 score: 0.45 f1 average: 0.706 accuracy: 0.928
train BCE loss:  0.18822012841701508  f1 score: 0.501 f1 average: 0.732 accuracy: 0.932
validation BCE loss:  0.18596026301383972  f1 score: 0.52 f1 average: 0.742 accuracy: 0.934
train BCE loss:  0.1749427169561386  f1 score: 0.558 f1 average: 0.762 accuracy: 0.937
validation BC

so the final scores on validation are  

validation BCE loss:  0.137 f1 score: 0.706 f1 average: 0.84 accuracy: 0.952

In [15]:
"""
now getting the results on the test set
"""

testpreds = torch.tensor([])
testtrues = torch.tensor([])
for batch in test_iterator:
    X = batch.comment_text
    y = batch.toxic
    testtrues = torch.cat((testtrues,y.cpu().detach()))
    pred = myTransformer(X).squeeze().cpu().detach()
    # print(valtrues.shape)
    testpreds = torch.cat((testpreds,pred))
err = F.binary_cross_entropy(testpreds,testtrues)
print("test BCE loss: ",err.item(),calculateMetrics(torch.round(testpreds).numpy(),testtrues.numpy()))
  

test BCE loss:  0.18907569348812103  f1 score: 0.646 f1 average: 0.804 accuracy: 0.932


In [17]:
test["predicted"] = torch.round(testpreds).numpy()


"""
this shows that the model understands the language well 

"""

test[test.predicted==1].iloc[32:37]


,comment_text,toxic,predicted
342,you fucking braindead oroszka 50% of the syria...,1,1.0
357,""" \r\n :::::""""This I hate Eagles 247 for. sinc...",1,1.0
370,cause she's just a devil with evil on her mind...,1,1.0
375,""" \r\n ::Can you cite any policies? If not, w...",1,1.0
392,"Agreed, it is stupid to say that any Malfoy is...",0,1.0
